In [2]:
vehicle_counter_DF = spark.read.csv('/home/amani/per-vehicle-records-2019-10-31.csv',inferSchema = True, header = True)

In [3]:
type(vehicle_counter_DF)

pyspark.sql.dataframe.DataFrame

In [4]:
vehicle_counter_DF.count()

4389299

In [5]:
vehicle_counter_DF.registerTempTable("vehicle_counter")

In [6]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
import time

In [15]:
def save(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, package=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="question1", keyspace="streaming")\
        .save(mode="append")
    except:pass

In [5]:
def save3(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, word=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="question3", keyspace="streaming")\
        .save(mode="append")
    except:pass

In [6]:
def save4(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, word=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="question4", keyspace="streaming")\
        .save(mode="append")
    except:pass

In [10]:
ssc = StreamingContext(sc, 5)

In [14]:
#Show total number of counts (on each site) by vehicle class.

result1=spark.sql("SELECT  count(class) as vehiclecount , class  from vehicle_counter  group by class order by count(class) desc ")
result1.show()


+------------+-----+
|vehiclecount|class|
+------------+-----+
|     3472965|    2|
|      498505|    3|
|      216978|    6|
|      135202|    5|
|       29347|    4|
|       21224|    7|
|       14682|    1|
|         396|    0|
+------------+-----+



In [7]:
#2. Compute the average speed (on each site) by vehicle class.

result2 = vehicle_counter_DF.groupBy("class")
result2.agg({'speed':'avg'}).show() 


+-----+-----------------+
|class|       avg(speed)|
+-----+-----------------+
|    1|75.41983381010762|
|    6|81.93572758528522|
|    3|90.35929148153001|
|    5|80.11806925933027|
|    4| 79.0626980611306|
|    7|  80.509602336977|
|    2|87.99111496948547|
|    0|81.18964646464646|
+-----+-----------------+



In [9]:
#3. Find the top 3 busiest counter sites in Ireland.
result3=spark.sql("SELECT cosit ,count(cosit) as cositcount from vehicle_counter group by cosit order by count(cosit) desc limit 3")
result3.show()

+-----+----------+
|cosit|cositcount|
+-----+----------+
| 1508|     98292|
| 1502|     89498|
| 1503|     86195|
+-----+----------+



In [10]:
#4. Find total number of counts for HGVs.

result4=spark.sql("SELECT  count(cosit) as HGVcount, cosit  from vehicle_counter  where classname in ('HGV_ART','HGV_RIG') group by cosit order by count(cosit) desc limit 1 ") 
result4.show()

+--------+-----+
|HGVcount|cosit|
+--------+-----+
|   12031|  997|
+--------+-----+



In [18]:
lines = ssc.textFileStream("file:///home/amani/streaming/")
records = lines.map(lambda line: line.split(","))



In [ ]:
result1 = records.map(lambda x: ((x[0],x[14],1))
result1 = result1.reduceByKey(lambda a,b : a+b)
result1.foreachRDD(save)
result1.pprint()
ssc.start()
time.sleep(300)
ssc.stop(stopSparkContext=False)